# 决策树分析：预测什么样的人可能是独生子

In [23]:
set.seed(42) # 最好的随机数种子
library(tidyverse)
library(rpart.plot)
library(rsample)
library(tidymodels)
library(yardstick)

In [2]:
YPS_file_path <- getwd() |> dirname() |> paste("/data/YoungPeopleSurvey/responses.csv",sep="") |> file.path()
YPS_dt <- read.csv(YPS_file_path) %>% mutate
dim(YPS_dt)
head(YPS_dt)

[1] 1010  150

,Music,Slow.songs.or.fast.songs,Dance,Folk,Country,Classical.music,Musical,Pop,Rock,Metal.or.Hardrock,...,Age,Height,Weight,Number.of.siblings,Gender,Left...right.handed,Education,Only.child,Village...town,House...block.of.flats
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,5,3,2,1,2,2,1,5,5,1,...,20,163,48,1,female,right handed,college/bachelor degree,no,village,block of flats
2,4,4,2,1,1,1,2,3,5,4,...,19,163,58,2,female,right handed,college/bachelor degree,no,city,block of flats
3,5,5,2,2,3,4,5,3,5,3,...,20,176,67,2,female,right handed,secondary school,no,city,block of flats
4,5,3,2,1,1,1,1,2,2,1,...,22,172,59,1,female,right handed,college/bachelor degree,yes,city,house/bungalow
5,5,3,4,3,2,4,3,5,3,1,...,20,170,59,1,female,right handed,secondary school,no,village,house/bungalow
6,5,3,2,3,2,3,3,2,5,5,...,20,186,77,1,male,right handed,secondary school,no,city,block of flats


查看数据类型

In [3]:
glimpse(YPS_dt)

Rows: 1,010
Columns: 150
$ Music                          <int> 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ~
$ Slow.songs.or.fast.songs       <int> 3, 4, 5, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, ~
$ Dance                          <int> 2, 2, 2, 2, 4, 2, 5, 3, 3, 2, 3, 1, 1, ~
$ Folk                           <int> 1, 1, 2, 1, 3, 3, 3, 2, 1, 5, 2, 1, 2, ~
$ Country                        <int> 2, 1, 3, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, ~
$ Classical.music                <int> 2, 1, 4, 1, 4, 3, 2, 2, 2, 2, 2, 4, 4, ~
$ Musical                        <int> 1, 2, 5, 1, 3, 3, 2, 2, 4, 5, 3, 1, 3, ~
$ Pop                            <int> 5, 3, 3, 2, 5, 2, 5, 4, 3, 3, 4, 2, 3, ~
$ Rock                           <int> 5, 5, 5, 2, 3, 5, 3, 5, 5, 5, 3, 5, 5, ~
$ Metal.or.Hardrock              <int> 1, 4, 3, 1, 1, 5, 1, 1, 5, 2, 2, 1, 4, ~
$ Punk                           <int> 1, 4, 4, 4, 2, 3, 1, 2, 1, 3, 1, 1, 2, ~
$ Hiphop..Rap                    <int> 1, 1, 1, 2, 5, 4, 3, 3, 1, 2, 3, 1, 3, ~
$ Reggae..Ska  

先看看NA的情况

In [4]:
YPS_dt %>% summary()

     Music       Slow.songs.or.fast.songs     Dance            Folk      
 Min.   :1.000   Min.   :1.000            Min.   :1.000   Min.   :1.000  
 1st Qu.:5.000   1st Qu.:3.000            1st Qu.:2.000   1st Qu.:1.000  
 Median :5.000   Median :3.000            Median :3.000   Median :2.000  
 Mean   :4.732   Mean   :3.328            Mean   :3.113   Mean   :2.289  
 3rd Qu.:5.000   3rd Qu.:4.000            3rd Qu.:4.000   3rd Qu.:3.000  
 Max.   :5.000   Max.   :5.000            Max.   :5.000   Max.   :5.000  
 NA's   :3       NA's   :2                NA's   :4       NA's   :5      
    Country      Classical.music    Musical           Pop       
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:1.000   1st Qu.:2.000   1st Qu.:2.000   1st Qu.:3.000  
 Median :2.000   Median :3.000   Median :3.000   Median :4.000  
 Mean   :2.123   Mean   :2.956   Mean   :2.762   Mean   :3.472  
 3rd Qu.:3.000   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.000  
 Max.   :5.000   M

清洗掉有NA的数据  
这里除了有numeric的NA，还有character的(nothing)  
na.omit可以删除NA，但不能删除(nothing)  
所以得另外处理一下  
方法为元素为""的情况就不选择

In [5]:
YPS_dt <- na.omit(YPS_dt) %>%  
            filter(if_any(everything(),~. %in% c("yes","no")),TRUE)
YPS_dt[,"Only.child"] %>% unique()
YPS_dt %>% summary()

[1] "no"  "yes"

     Music       Slow.songs.or.fast.songs     Dance            Folk      
 Min.   :1.000   Min.   :1.000            Min.   :1.000   Min.   :1.000  
 1st Qu.:5.000   1st Qu.:3.000            1st Qu.:2.000   1st Qu.:1.000  
 Median :5.000   Median :3.000            Median :3.000   Median :2.000  
 Mean   :4.759   Mean   :3.295            Mean   :3.073   Mean   :2.258  
 3rd Qu.:5.000   3rd Qu.:4.000            3rd Qu.:4.000   3rd Qu.:3.000  
 Max.   :5.000   Max.   :5.000            Max.   :5.000   Max.   :5.000  
    Country      Classical.music    Musical           Pop       
 Min.   :1.000   Min.   :1.00    Min.   :1.000   Min.   :1.000  
 1st Qu.:1.000   1st Qu.:2.00    1st Qu.:2.000   1st Qu.:3.000  
 Median :2.000   Median :3.00    Median :3.000   Median :4.000  
 Mean   :2.111   Mean   :2.98    Mean   :2.759   Mean   :3.439  
 3rd Qu.:3.000   3rd Qu.:4.00    3rd Qu.:4.000   3rd Qu.:4.000  
 Max.   :5.000   Max.   :5.00    Max.   :5.000   Max.   :5.000  
      Rock       Metal.or.H

In [6]:
YPS_dt <- YPS_dt %>% 
    mutate_if(is.character,factor)
glimpse(YPS_dt)

Rows: 685
Columns: 150
$ Music                          <int> 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ~
$ Slow.songs.or.fast.songs       <int> 3, 4, 5, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, ~
$ Dance                          <int> 2, 2, 2, 4, 2, 5, 3, 2, 3, 1, 1, 5, 2, ~
$ Folk                           <int> 1, 1, 2, 3, 3, 3, 2, 5, 2, 1, 2, 3, 1, ~
$ Country                        <int> 2, 1, 3, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, ~
$ Classical.music                <int> 2, 1, 4, 4, 3, 2, 2, 2, 2, 4, 4, 1, 2, ~
$ Musical                        <int> 1, 2, 5, 3, 3, 2, 2, 5, 3, 1, 3, 5, 3, ~
$ Pop                            <int> 5, 3, 3, 5, 2, 5, 4, 3, 4, 2, 3, 5, 4, ~
$ Rock                           <int> 5, 5, 5, 3, 5, 3, 5, 5, 3, 5, 5, 2, 5, ~
$ Metal.or.Hardrock              <int> 1, 4, 3, 1, 5, 1, 1, 2, 2, 1, 4, 1, 2, ~
$ Punk                           <int> 1, 4, 4, 2, 3, 1, 2, 3, 1, 1, 2, 1, 5, ~
$ Hiphop..Rap                    <int> 1, 1, 1, 5, 4, 3, 3, 2, 3, 1, 3, 2, 3, ~
$ Reggae..Ska    

看看大概分别有多少样本，感觉还行，，，

In [7]:
YPS_dt %>%
    group_by(Only.child) %>%
    summarize(n=n())

Only.child,n
<fct>,<int>
no,526
yes,159


In [8]:
unique(YPS_dt$Only.child)

[1] no  yes
Levels: no yes

将数据集分为训练集和测试集

In [9]:
YPS_split <- initial_split(YPS_dt,strata = Only.child,prop=0.75)
YPS_training <- training(YPS_split)
YPS_testing <- testing(YPS_split)

通过比例确定数据集的层化抽样是成功的

In [10]:
YPS_training %>%
    group_by(Only.child) %>%
    summarize(n=n(),prop=n()/nrow(YPS_training))

YPS_testing %>%
    group_by(Only.child) %>%
    summarize(n=n(),prop=n()/nrow(YPS_testing))

Only.child,n,prop
<fct>,<int>,<dbl>
no,394,0.7680312
yes,119,0.2319688


Only.child,n,prop
<fct>,<int>,<dbl>
no,132,0.7674419
yes,40,0.2325581


In [11]:
YPS_spec <- decision_tree(
  tree_depth=tune(), # 树的深度
  min_n=tune(), # 最小分裂要求的数量
  cost_complexity=tune() # 
  ) %>%
  set_engine("rpart") %>% # CART用rpart
  set_mode("classification") # 因为是分类变量所以用classification

YPS_spec

Decision Tree Model Specification (classification)

Main Arguments:
  cost_complexity = tune()
  tree_depth = tune()
  min_n = tune()

Computational engine: rpart 


In [12]:
YPS_fold <- vfold_cv(YPS_training,v=5)
YPS_grid <- grid_regular(parameters(YPS_spec),levels=3)

Warning message:
"`parameters.model_spec()` was deprecated in tune 0.1.6.9003.
i Please use `hardhat::extract_parameter_set_dials()` instead."


In [13]:
YPS_spec_tuned_para <- tune_grid(
    YPS_spec,
    Only.child~.,
    resample=YPS_fold,
    grid=YPS_grid,
    metrics=metric_set(accuracy) 
)

In [15]:
YPS_spec_tuned_para_best <- select_best(YPS_spec_tuned_para)
YPS_spec_tuned_para_best

cost_complexity,tree_depth,min_n,.config
<dbl>,<int>,<int>,<chr>
1e-10,1,2,Preprocessor1_Model01


In [16]:
YPS_spec_tuned <- finalize_model(
    YPS_spec,
    YPS_spec_tuned_para_best
)

In [37]:
YPS_training_fitted <- fit(
    YPS_spec_tuned,
    formula=Only.child~.,
    data=YPS_training
)
YPS_training_fitted

parsnip model object

n= 513 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 513 119 no (0.76803119 0.23196881)  
  2) Number.of.siblings>=0.5 435  41 no (0.90574713 0.09425287) *
  3) Number.of.siblings< 0.5 78   0 yes (0.00000000 1.00000000) *

In [96]:
YPS_prediction <- predict(
    YPS_training_fitted,
    new_data=YPS_testing
)
YPS_prediction

.pred_class
<fct>
no
no
no
no
no
no
no
no
no


In [30]:
YPS_pre_combind <- YPS_prediction %>% 
    mutate(ture_result=YPS_testing$Only.child)
YPS_est_mtx <- conf_mat(
    YPS_pre_combind, 
    estimate=.pred_class,
    truth=ture_result
)
YPS_est_mtx

accuracy(
    YPS_pre_combind, 
    estimate=.pred_class,
    truth=ture_result
    )

          Truth
Prediction  no yes
       no  130  19
       yes   2  21

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.877907


直接上rpart试试

In [99]:
model <- rpart(Only.child~.,data=YPS_training)


pred <- predict(
    model,
    newdata=YPS_testing,
    type="class"
) %>% data.frame(pred=.)
pred

,pred
,<fct>
7,no
8,no
11,no
13,no
15,no
18,no
19,no
22,no
25,no


In [101]:
comb <- pred %>% mutate(ture_result=YPS_testing$Only.child)
matrix <- conf_mat(
    comb, 
    estimate=pred,
    truth=ture_result
)
YPS_est_mtx

accuracy(
    comb, 
    estimate=pred,
    truth=ture_result
    )

          Truth
Prediction  no yes
       no  130  19
       yes   2  21

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.8662791


结果是：还是用vfold_cv tune一下参数比较好，虽然也就高个1.1%吧，，，